In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from mtl_pytorch.layer_node import Conv2dNode, BN2dNode

from mtl_pytorch.trainer import Trainer
from data.heads.pixel2pixel import ASPPHeadNode

from data.dataloader.cityscapes_dataloader import CityScapes
from data.metrics.pixel2pixel_loss import CityScapesCriterions
from data.metrics.pixel2pixel_metrics import CityScapesMetrics

from mobilenetv2 import mobilenet_v2

from mtl_model import mtl_model

from data.dataloader.nyuv2_dataloader import NYU_v2
from data.metrics.pixel2pixel_loss import NYUCriterions
from data.metrics.pixel2pixel_metrics import NYUMetrics

from data.dataloader.taskonomy_dataloader import Taskonomy
from data.metrics.pixel2pixel_loss import TaskonomyCriterions
from data.metrics.pixel2pixel_metrics import TaskonomyMetrics

# Data

In [2]:
dataroot = 'datasets/Cityscapes/'

headsDict = nn.ModuleDict()
trainDataloaderDict = {}
valDataloaderDict = {}
criterionDict = {}
metricDict = {}

tasks = ['segment_semantic', 'depth_zbuffer']
task_cls_num = {'segment_semantic': 19, 'depth_zbuffer': 1}
for task in tasks:
    headsDict[task] = ASPPHeadNode(1280, task_cls_num[task])

    # For model trainer
    dataset = CityScapes(dataroot, 'train', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    dataset = CityScapes(dataroot, 'test', task)
    valDataloaderDict[task] = DataLoader(dataset, 16, shuffle=True)

    criterionDict[task] = CityScapesCriterions(task)
    metricDict[task] = CityScapesMetrics(task)

# Model

In [3]:
mtlmodel = mobilenet_v2(False, heads_dict=headsDict)
# don't use wrapper anymore mtlmodel = mtl_model(model=model, headsDict=headsDict)

# Train

In [4]:
checkpoint = 'checkpoint/'
trainer = Trainer(mtlmodel, trainDataloaderDict, valDataloaderDict, criterionDict, metricDict, 
                  print_iters=100, val_iters=500, save_num=1, policy_update_iters=100)

### pre_train

In [5]:
trainer.pre_train(iters=1, lr=0.0001, savePath=checkpoint+'Cityscapes/')

0
segment_semantic
before forwarding pre_train_all
depth_zbuffer
before forwarding pre_train_all


### alter_train

In [6]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1, 'policy':0.0005}
trainer.alter_train_with_reg(iters=4, policy_network_iters=(100,400), policy_lr=0.01, network_lr=0.0001,
                             loss_lambda=loss_lambda,
                             savePath=checkpoint+'Cityscapes/')

before forwarding mtl
before forwarding mtl
before forwarding mtl
before forwarding mtl
before forwarding mtl
before forwarding mtl
before forwarding mtl
before forwarding mtl


### sample policy from trained policy distribution and save

In [7]:
policy_list = {'segment_semantic': [], 'depth_zbuffer': []}
name_list = {'segment_semantic': [], 'depth_zbuffer': []}

In [8]:
for name, param in mtlmodel.named_parameters():
    if 'policy' in name and not torch.eq(param, torch.tensor([0., 0., 0.])).all():
        if 'segment_semantic' in name:
            policy_list['segment_semantic'].append(param.data.cpu().detach().numpy())
            name_list['segment_semantic'].append(name)
        elif 'depth_zbuffer' in name:
            policy_list['depth_zbuffer'].append(param.data.cpu().detach().numpy())
            name_list['depth_zbuffer'].append(name)

features.models.0.models.0.policy.segment_semantic Parameter containing:
tensor([ 0.0107, -0.0093, -0.0263], requires_grad=True)
features.models.0.models.0.policy.depth_zbuffer Parameter containing:
tensor([ 0.0501, -0.0490, -0.0488], requires_grad=True)
features.models.18.models.0.policy.segment_semantic Parameter containing:
tensor([-4.8038e-02, -1.8133e-05,  4.0110e-02], requires_grad=True)
features.models.18.models.0.policy.depth_zbuffer Parameter containing:
tensor([-0.0368,  0.0267,  0.0192], requires_grad=True)


In [9]:
from collections import OrderedDict
from scipy.special import softmax

sample_policy_dict = OrderedDict()
for task in tasks:
    for name, policy in zip(name_list[task], policy_list[task]):
        # distribution = softmax(policy, axis=-1
        distribution = softmax(policy, axis=-1)
        distribution /= sum(distribution)

        choice = np.random.choice((0,1,2), p=distribution)
        if choice == 0:
            sample_policy_dict[name] = torch.tensor([1.0,0.0,0.0])
        elif choice == 1:
            sample_policy_dict[name] = torch.tensor([0.0,1.0,0.0])
        elif choice == 2:
            sample_policy_dict[name] = torch.tensor([0.0,0.0,1.0])

In [10]:
print(sample_policy_dict)

OrderedDict([('features.models.0.models.0.policy.segment_semantic', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.segment_semantic', tensor([0., 0., 1.])), ('features.models.0.models.0.policy.depth_zbuffer', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.depth_zbuffer', tensor([0., 1., 0.]))])


In [18]:
sample_path = '.'
# app.run(debug=True, use_reloader=False)
sample_state = {'state_dict': sample_policy_dict}
torch.save(sample_state, sample_path + 'sample_policy.model')

### post train from scratch

In [24]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1}
trainer.post_train(iters=1, lr=0.001,
                   decay_lr_freq=4000, decay_lr_rate=0.5,
                   loss_lambda=loss_lambda,
                   savePath=checkpoint+'Cityscapes/',
                   reload='sample_policy.model')

Task Policy:
OrderedDict([('features.models.0.models.0.policy.segment_semantic', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.segment_semantic', tensor([0., 0., 1.])), ('features.models.0.models.0.policy.depth_zbuffer', tensor([1., 0., 0.])), ('features.models.18.models.0.policy.depth_zbuffer', tensor([0., 1., 0.]))])
segment_semantic
depth_zbuffer


### get the validation results in the paper 

In [25]:
mtlmodel.load_state_dict(torch.load('CityScapes.model'))
trainer.validate('mtl', hard=True) 

FileNotFoundError: [Errno 2] No such file or directory: 'CityScapes.model'

# Inference

In [ ]:
mtlmodel.load_state_dict(torch.load('CityScapes.model'))
output = mtlmodel(x, task='segment_semantic', hard=True)